This notebook will serve as the master notebook which the TAs can run from start to finish

In [8]:
import pandas as pd

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Define the path to the data
data_path = './data/essay_df.pkl'
df = pd.read_pickle(data_path)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 29 columns):
essay_id          12976 non-null int64
essay_set         12976 non-null int64
essay             12976 non-null object
rater1_domain1    12976 non-null int64
rater2_domain1    12976 non-null int64
rater3_domain1    128 non-null float64
domain1_score     12976 non-null int64
rater1_domain2    1800 non-null float64
rater2_domain2    1800 non-null float64
domain2_score     1800 non-null float64
rater1_trait1     2292 non-null float64
rater1_trait2     2292 non-null float64
rater1_trait3     2292 non-null float64
rater1_trait4     2292 non-null float64
rater1_trait5     723 non-null float64
rater1_trait6     723 non-null float64
rater2_trait1     2292 non-null float64
rater2_trait2     2292 non-null float64
rater2_trait3     2292 non-null float64
rater2_trait4     2292 non-null float64
rater2_trait5     723 non-null float64
rater2_trait6     723 non-null float64
rater3_trait1     128

In [19]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12976 entries, 0 to 12975
Data columns (total 29 columns):
essay_id          12976 non-null int64
essay_set         12976 non-null int64
essay             12976 non-null object
rater1_domain1    12976 non-null int64
rater2_domain1    12976 non-null int64
rater3_domain1    128 non-null float64
domain1_score     12976 non-null int64
rater1_domain2    1800 non-null float64
rater2_domain2    1800 non-null float64
domain2_score     1800 non-null float64
rater1_trait1     2292 non-null float64
rater1_trait2     2292 non-null float64
rater1_trait3     2292 non-null float64
rater1_trait4     2292 non-null float64
rater1_trait5     723 non-null float64
rater1_trait6     723 non-null float64
rater2_trait1     2292 non-null float64
rater2_trait2     2292 non-null float64
rater2_trait3     2292 non-null float64
rater2_trait4     2292 non-null float64
rater2_trait5     723 non-null float64
rater2_trait6     723 non-null float64
rater3_trait1     128

In [21]:
df.memory_usage(index=True, deep=True).sum()

20896129

In [ ]:
# Define a batch generator
# Adopted from HW3
def get_batches(essays, scores, n_essays):
    '''
    Partition data array into mini-batches
    param essays: array of essays in word embedding form
    param scores: array of scores of essays
    param n_essays: number of essays in a mini-batch
    output:
    '''
    batch_size = n_seqs * n_steps
    n_batches = int(len(array) / batch_size)
    
    # we only keep the full batches and ignore the rest.

    array = array[:batch_size * n_batches]
    array = array.reshape((n_seqs, -1))

    # Label array is created by rolling the regular array by 1 along
    # the rows.
    label_array = np.roll(array, -1)
    
    # Loop to generate batches for inputs and targets
    while True:
        for i in range(n_batches):
            batch_x = array[:,i*n_steps: (i+1)*n_steps]
            batch_y = label_array[:,i*n_steps : (i+1)*n_steps]

            yield (batch_x, batch_y)